In [2]:
# IMPORT NECESSARY MODULES
import os
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import heapq
import math
import time

folder_path = "D:\\UPD Files\\BS CS 24-25 1st Sem\\CS 198\\cs198-protein-folding\\datasets"

In [ ]:
import time
import math

def bellman_ford_wrap_around_surface(surface, start, goal, alpha=0):
    """
    Bellman-Ford algorithm on a wrap-around energy surface.

    Parameters:
    - surface: 2D list representing energy values at each grid point.
    - start: (x, y) tuple for the starting position.
    - goal: (x, y) tuple for the goal position.
    - alpha: Weighting factor for the energy cost component.
    """
    rows, cols = len(surface), len(surface[0])
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]

    def get_neighbors(node):
        x, y = node
        return [((x + dx) % rows, (y + dy) % cols) for dx, dy in directions]

    def energy_cost(current, neighbor):
        """Weighted cost function considering wrap-around distances and energy."""
        x1, y1 = current
        x2, y2 = neighbor
        
        # Compute wrap-around distances
        dx = min(abs(x1 - x2), rows - abs(x1 - x2))
        dy = min(abs(y1 - y2), cols - abs(y1 - y2))

        # Use energy at the neighbor node as cost
        energy_term = alpha * surface[x2][y2]

        # Weighted cost function: spatial distance + weighted energy term
        return math.sqrt(dx ** 2 + dy ** 2) + energy_term  

    # Step 1: Initialize distances
    distance = { (x, y): float("inf") for x in range(rows) for y in range(cols) }
    distance[start] = 0
    came_from = {}

    nodes_explored = 0
    start_time = time.time()

    # Step 2: Relax all edges (V-1 times)
    for _ in range(rows * cols - 1):
        updated = False
        for x in range(rows):
            for y in range(cols):
                current = (x, y)
                if distance[current] == float("inf"):
                    continue
                for neighbor in get_neighbors(current):
                    new_cost = distance[current] + energy_cost(current, neighbor)
                    if new_cost < distance[neighbor]:
                        distance[neighbor] = new_cost
                        came_from[neighbor] = current
                        updated = True
                        nodes_explored += 1
        if not updated:
            break  # Stop early if no updates occur

    # Step 3: Check for negative weight cycles (not needed here since distances are always positive)

    # Reconstruct path
    if distance[goal] == float("inf"):
        print("No path found.")
        return None

    path = []
    current = goal
    while current in came_from:
        path.append(current)
        current = came_from[current]
    path.append(start)
    path.reverse()

    execution_time = time.time() - start_time
    print("Bellman-Ford Total cost:", distance[goal])
    print("Path length:", len(path))
    print("Execution time (s):", execution_time)
    print("Nodes explored:", nodes_explored)
    
    return path
